In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.ensemble import RandomForestClassifier

In [2]:
train_data = pd.read_csv("csci-3360-hw3/train.csv", low_memory=False)
training_data = train_data.drop('BRIDGE_CONDITION', axis = 1)
y_train = train_data['BRIDGE_CONDITION']
testing_data = pd.read_csv("csci-3360-hw3/test.csv", low_memory=False)

bridge_ids = testing_data['BRIDGE_ID'].copy()

training_data['BRIDGE_AGE'] = 2024 - training_data['YEAR_BUILT_027']
testing_data['BRIDGE_AGE'] = 2024 - testing_data['YEAR_BUILT_027']

training_data['TRAFFIC_CHANGE'] = training_data['FUTURE_ADT_114'] - training_data['ADT_029']
testing_data['TRAFFIC_CHANGE'] = testing_data['FUTURE_ADT_114'] - testing_data['ADT_029']

columns_to_keep = ['TRAFFIC_CHANGE', 'FUTURE_ADT_114', 'BRIDGE_AGE', 'TRAFFIC_LANES_ON_028A', 'ADT_029', 'DESIGN_LOAD_031', 'MAINTENANCE_021', 'DECK_COND_058', 'SUPERSTRUCTURE_COND_059', 'SUBSTRUCTURE_COND_060', 'STRUCTURAL_EVAL_067', 'SCOUR_CRITICAL_113', 'YEAR_RECONSTRUCTED_106', 'OPERATING_RATING_064', 'INVENTORY_RATING_066']
training_data = training_data[columns_to_keep]
testing_data = testing_data[columns_to_keep]







In [3]:
training_data.head()

,TRAFFIC_CHANGE,FUTURE_ADT_114,BRIDGE_AGE,TRAFFIC_LANES_ON_028A,ADT_029,DESIGN_LOAD_031,MAINTENANCE_021,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,STRUCTURAL_EVAL_067,SCOUR_CRITICAL_113,YEAR_RECONSTRUCTED_106,OPERATING_RATING_064,INVENTORY_RATING_066
0,-100,120,89,2,220,0,2,5,4,6,4,U,NaN,35.3,20.8
1,1335,4005,75,2,2670,2,1,6,6,6,6,5,1995.0,40.8,24.4
2,2540,27540,47,2,25000,6,1,N,N,N,7,8,NaN,55.3,32.6
3,323,11073,56,2,10750,5,1,5,5,6,5,5,0.0,28.2,20.1
4,-3860,900,45,2,4760,3,2,7,7,7,7,U,NaN,70.7,42.6


In [4]:
testing_data.head()



,TRAFFIC_CHANGE,FUTURE_ADT_114,BRIDGE_AGE,TRAFFIC_LANES_ON_028A,ADT_029,DESIGN_LOAD_031,MAINTENANCE_021,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,STRUCTURAL_EVAL_067,SCOUR_CRITICAL_113,YEAR_RECONSTRUCTED_106,OPERATING_RATING_064,INVENTORY_RATING_066
0,891,2895,64,2,2004,2,2,7,7,7,6,U,0.0,35.3,24.4
1,891,2895,10,2,2004,5,2,7,7,8,7,U,NaN,53.5,31.7
2,1,3,27,1,2,5,64,7,7,7,6,5,2004.0,29.5,22.7
3,-56,3000,14,2,3056,6,2,7,7,7,7,U,NaN,65.3,29.0
4,1030,2970,19,2,1940,6,1,7,7,7,7,8,NaN,55.3,32.6


In [5]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(training_data)
X_train_encoded = encoder.transform(training_data)
X_test_encoded = encoder.transform(testing_data)





In [6]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train_encoded, y_train)
predictions = model.predict(X_test_encoded)

output = pd.DataFrame({'BRIDGE_ID': bridge_ids, 'BRIDGE_CONDITION': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [7]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train_encoded, y_train, cv=5)
print("Individual accuracy:", scores)
print("Mean accuracy:", scores.mean())

Individual accuracy: [0.75371655 0.75569871 0.75421209 0.75359445 0.75409023]
Mean accuracy: 0.7542624068067609
